In [10]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
import functools
import concurrent.futures
from numba import njit
import tensorflow as tf
import scipy
import sparse
import itertools
import linecache
from multiprocessing import Pool, cpu_count
import sympy

# Generación de base
class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)[::-1]  

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)

    def idx_to_repr(self, idx):
        return self.canonicals[idx]

    def opr_to_idx(self, opr):
        for i in range(self.size): # Evitar esto ordenando opr
            if self.base[i] == opr:
                return i
                
    def op_to_idx(self, op):
        # Normalizamos el orden
        op = of.transforms.normal_ordered(op)
        # Si es nulo, devolvemos None
        if len(op.terms) == 0:
            return None
        act_lvls = lambda tt: [tpl[0] for tpl in next(iter(tt.terms.keys()))] # Consistente con lo anterior
        act = act_lvls(op)
        act_to_int = lambda x: np.sum([2**i for i in x])
        search = np.where(self.num_ele == act_to_int(act))[0]
        if search.size == 0:
            return None
        else:
            return search[0]

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None, pairs = False):
        basis = []
        num_ele = []
        for k in range(0,2**d):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    if pairs:
                        if np.all(b[::2] == b[1::2]):
                            oper = self.bin_to_op(b)
                            basis.append(oper)
                            num_ele.append(k)
                    else:
                        oper = self.bin_to_op(b)
                        basis.append(oper)
                        num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def signs_gen(self, base):
        signs = []
        for op in base:
            op_normal = of.transforms.normal_ordered(op)
            if op_normal.terms:
                coeff = next(iter(op_normal.terms.values()))
                signs.append(np.sign(coeff))
            else:
                signs.append(0)
        return np.array(signs)

    def __init__(self, d, num = None, pairs = False, basis = None, num_ele = None):
        self.d = d
        self.num = num
        self.m = num
        # Si nos da la base, la levantamos (asumimos GC). Si no, la creamos
        if basis is None:
            self.base, self.num_ele = self.create_basis(d, num, pairs)
            self.signs = self.signs_gen(self.base)
        else:
            self.base, self.num_ele = basis, num_ele
            self.signs = self.signs_gen(basis)
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        self.pairs = pairs

    @staticmethod
    def cdc(i, j):
        return of.FermionOperator(((i,1),(j,0)))

    @staticmethod
    def cc(i, j):
        return of.FermionOperator(((i,0),(j,0)))

    # Del indice, cuenta el número de partículas
    def num_idx(self, idx):
        b = self.int_to_bin(idx, basis.d)
        return b.count('1')

    # Calculo de rho1 (via directa, lento, y solo definido en la base por ahora)
    def rho_1(self, op):
        # Necesitamos un índice, es?
        if type(op) != int:
            op = self.opr_to_idx(op)
        mat = np.zeros((self.d, self.d))
        for i in range(self.d):
            for j in range(self.d):
                cdc = self.cdc(j, i)
                mat[i,j] = self.idx_mean_val(op, cdc)
        return mat

# Calculo de generadores de rho1
def rho_1_gen(basis):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    indices = []
    values = []
    shape = (basis.d, basis.d, basis.size, basis.size)
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            # Generamos el operador
            op = basis.cdc(j, i)
            #print(op)
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# Calculo de rho1 (via generadores) de un vector en la base canonica
def rho_1(vect, rho_1_arrays):
    if len(vect.shape) == 1: # vectores
        return sparse.einsum('k,ijkl,l->ij', vect, rho_1_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad
        return sparse.einsum('ijkl,kl->ij', rho_1_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_1_arrays)

# Calculo de indices de rho2kkbar
def get_kkbar_indices(t_basis):
    indices = []
    for i, ind in enumerate(t_basis.num_ele):
        v = t_basis.int_to_bin(ind, t_basis.d)
        if np.all(v[::2] == v[1::2]):
            indices.append(i)
    return indices

# Calculo de generadores de rho2
def rho_2_gen(basis, t_basis, idx_list = []):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    d = basis.d
    indices = []
    values = []
    if len(idx_list) == basis.m:
        idx_list = idx_list
    elif len(idx_list) == basis.m**4:
        idx_list = np.unique(idx_list[:,0])
    else:
        idx_list = range(t_basis.size)
    shape = (len(idx_list), len(idx_list), basis.size, basis.size)
    for i, ii in tqdm(enumerate(idx_list), total=len(idx_list)):
        for j, jj in enumerate(idx_list):
            # Generamos el operador
            op = t_basis.base[jj]*of.utils.hermitian_conjugated(t_basis.base[ii])
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# rho_m_gen aux func
def process_chunk(args):
    chunk, m_basis, basis, d, it_set = args
    indices = []
    values = []
    for ii in chunk:
        for jj in it_set:
            # Generate the operator
            op = m_basis.base[jj] * of.utils.hermitian_conjugated(m_basis.base[ii])
            mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extract the information
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([ii, jj, r, c])
                values.append(v)
    return indices, values

# Parallelized rho_m_gen
def rho_m_gen(basis, m, num_workers=None):
    if num_workers is None:
        num_workers = cpu_count()  # Use all available CPUs by default
    
    indices = []
    values = []
    m_basis = fixed_basis(basis.d, num=m, pairs=basis.pairs)
    shape = (m_basis.size, m_basis.size, basis.size, basis.size)

    it_set = np.arange(m_basis.size)
    chunks = np.array_split(it_set, num_workers)  # Split `it_set` into chunks for each worker

    # Use multiprocessing Pool for parallel processing
    with Pool(processes=num_workers) as pool:
        # Pass arguments as tuples instead of using a lambda
        results = list(
            tqdm(
                pool.imap(
                    process_chunk, 
                    [(chunk, m_basis, basis, basis.d, it_set) for chunk in chunks]
                ),
                total=num_workers
            )
        )
    
    # Collect results from all processes
    for indices_chunk, values_chunk in results:
        indices.extend(indices_chunk)
        values.extend(values_chunk)

    # Construct the sparse array
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m(vect, rho_m_arrays):
    if len(vect.shape) == 1:
        return sparse.einsum('k,ijkl,l->ij', vect, rho_m_arrays, vect)
    else:
        return sparse.einsum('kl,ijlk->ij', vect, rho_m_arrays)

# Calculo de rho2 (via generadores) de un estado en la base canonica
def rho_2(vect, rho_2_arrays):
    if len(vect.shape) == 1: # vectores SOLO RHO2 COMPLETA
        return sparse.einsum('k,ijkl,l->ij', vect, rho_2_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad SOLO RHO2 COMPLETA
        return sparse.einsum('ijkl,kl->ij', rho_2_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_2_arrays)

# Calculo de generadores de K (usado para quasiparticles) WIP SPARSE
def k_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d), total=d):
        for j in range(0, d):
            op = basis.cc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

def k_vect(vect, k_gen):
    return np.einsum('k,ijkl,l->ij', vect, k_gen, vect)

# Calculo la matrix rho de cuasipartículas  WIP SPARSE
def rho_qsp(vect, rho_1_arrays, k_arrays, rho1 = None):
    if type(rho1) == None:
        rho1 = rho_1(vect, rho_1_arrays)
    k = k_vect(vect, k_arrays)

    mat = np.block([[rho1, k], [-np.conjugate(k), np.eye(rho_1_arrays.shape[0])-np.conjugate(rho1)]])
    return mat

# Devuelve los indices que tienen a level ocupado
def level_proy(d, level):
    ids = []
    for k in range(0,2**d):
        b = fixed_basis.int_to_bin(k, d)
        if b[level] == '1':
            ids.append(k)
    arr = np.zeros(2**d)
    arr[np.array(ids)] = 1
    return arr, ids

def parity_levels(d):
    rng = range(2**d)
    binary_repr = np.vectorize(np.binary_repr)(rng)
    ones_c = np.char.count(binary_repr, '1')
    return np.array(rng)[ones_c % 2 == 1] # seleccionamos estados impares

# Devuelve el vector postmedido
def measure(basis, vect, level = 1):
    l_arr, l_ids = level_proy(basis.d, level)
    proy_v = vect * l_arr
    comp_arr = np.logical_not(l_arr).astype(int)
    comp_v = vect * comp_arr
    norm = lambda v: v / np.linalg.norm(v)
    return norm(proy_v), norm(comp_v)

def entropy(rho, m):
    S_fun = lambda rho: -1*np.trace(rho @ scipy.linalg.logm(rho)) / np.log(2)
    ent = S_fun(rho) / (np.log2(scipy.special.binom(basis.d, m)))
    return ent

def build_csv_basis(csvf, d, m, split=True):
    # En caso de split, solo toma la mitad de cada estado (UP). Para la reconstrucción, asumieremos que la secuencia es la siguiente
    # UP1 X DOWN1, UP1 X DOWN2, ..., UP1 X DOWNN, UP2 X DOWN1... 
    # Construimos la base
    ops = []
    ops_int = []
    d = d//2 if split else d
    with open(csvf, 'r') as basis:
        num_ele = []
        # Contar los niveles
        m_level = 0
        # Creamos operadores
        for l in basis.read().splitlines()[4:]:
            natop = [int(x) for x in l.split(' ')[1:m//2+1 if split else None]] # Operador en forma de lista
            #print(natop)
            op = of.FermionOperator(([(i, 1) for i in natop]))
            # Contamos niveles
            m_level = max(m_level, *natop)
            # Determinamos el índice
            natop_to_int = lambda x: np.sum([2**i for i in x])
            naint = natop_to_int(natop)
            if naint not in ops_int:
                num_ele.append(naint)
                ops_int.append(naint)
                ops.append(op)

        # Determinamos m y d
        num = len(natop)
        #assert d == m_level+1
    # Para ser consistente con el orden de fixed_basis, reordenamos
    idx_set = np.argsort(num_ele)
    return fixed_basis(d, num = num, pairs = False, basis = np.array(ops)[idx_set], num_ele=np.array(num_ele)[idx_set]), idx_set

    # Al igual que rho_2_gen, genera el bloque c^d_i c^d_/bar{j} c_/bar{k} c_l
def block_process_chunk(args):
    chunk, basis, it_set = args
    indices = []
    values = []
    for idx1 in chunk:
        for idx2 in it_set:
            i, j = int(idx1 // basis.m), int(idx1 % basis.m)
            k, l = int(idx2 // basis.m), int(idx2 % basis.m)
            op1 = of.FermionOperator(((2*k, 1), (2*l+1, 1)))
            op2 = of.FermionOperator(((2*j+1, 0), (2*i, 0)))
            mat = np.real(of.get_sparse_operator( op1 * op2, n_qubits=basis.d)[np.ix_(basis.num_ele, basis.num_ele)])
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([idx1, idx2, r, c])
                values.append(v)
    return indices, values

def rho_2_block_gen(basis, m, num_workers=None):
    if num_workers is None:
        num_workers = cpu_count()  # Use all available CPUs by default

    indices = []
    values = []
    shape = (basis.m**2, basis.m**2, basis.size, basis.size)

    it_set = np.arange(basis.m**2)
    chunks = np.array_split(it_set, num_workers)  # Split `it_set` into chunks for each worker

    # Use multiprocessing Pool for parallel processing
    with Pool(processes=num_workers) as pool:
        # Pass arguments as tuples instead of using a lambda
        results = list(
            tqdm(
                pool.imap(
                    block_process_chunk,
                    [(chunk, basis, it_set) for chunk in chunks]
                ),
                total=num_workers
            )
        )

    # Collect results from all processes
    for indices_chunk, values_chunk in results:
        indices.extend(indices_chunk)
        values.extend(values_chunk)

    # Construct the sparse array
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# rho_m_gen aux func
def process_chunk(args):
    chunk, m_basis, basis, d, it_set = args
    indices = []
    values = []
    for ii in chunk:
        for jj in it_set:
            # Generate the operator
            op = m_basis.base[jj] * of.utils.hermitian_conjugated(m_basis.base[ii])
            mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extract the information
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([ii, jj, r, c])
                values.append(v)
    return indices, values

# Parallelized rho_m_gen
def rho_m_gen(basis, m, num_workers=None):
    if num_workers is None:
        num_workers = cpu_count()  # Use all available CPUs by default

    indices = []
    values = []
    m_basis = fixed_basis(basis.d, num=m)
    shape = (m_basis.size, m_basis.size, basis.size, basis.size)

    it_set = np.arange(m_basis.size)
    chunks = np.array_split(it_set, num_workers)  # Split `it_set` into chunks for each worker

    # Use multiprocessing Pool for parallel processing
    with Pool(processes=num_workers) as pool:
        # Pass arguments as tuples instead of using a lambda
        results = list(
            tqdm(
                pool.imap(
                    process_chunk,
                    [(chunk, m_basis, basis, basis.d, it_set) for chunk in chunks]
                ),
                total=num_workers
            )
        )

    # Collect results from all processes
    for indices_chunk, values_chunk in results:
        indices.extend(indices_chunk)
        values.extend(values_chunk)

    # Construct the sparse array
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def kkbar_process_chunk(args):
    chunk, basis, it_set = args
    indices = []
    values = []
    for ii in chunk:
        for jj in it_set:
            ii, jj = int(ii), int(jj)
            op1 = of.FermionOperator(((2*jj, 1), (2*jj+1, 1)))
            op2 = of.FermionOperator(((2*ii+1, 0), (2*ii, 0)))
            mat = np.real(of.get_sparse_operator(op1 * op2, n_qubits=basis.d)[np.ix_(basis.num_ele, basis.num_ele)])
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([ii, jj, r, c])
                values.append(v)
    return indices, values

def rho_2_kkbar_gen(basis, num_workers=None):
    if num_workers is None:
        num_workers = cpu_count()  # Use all available CPUs by default

    indices = []
    values = []
    shape = (basis.m, basis.m, basis.size, basis.size)

    it_set = np.arange(basis.m)
    chunks = np.array_split(it_set, num_workers)  # Split `it_set` into chunks for each worker

    # Use multiprocessing Pool for parallel processing
    with Pool(processes=num_workers) as pool:
        # Pass arguments as tuples instead of using a lambda
        results = list(
            tqdm(
                pool.imap(
                    kkbar_process_chunk,
                    [(chunk, basis, it_set) for chunk in chunks]
                ),
                total=num_workers
            )
        )

    # Collect results from all processes
    for indices_chunk, values_chunk in results:
        indices.extend(indices_chunk)
        values.extend(values_chunk)

    # Construct the sparse array
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

### Old

In [11]:
d = 8
num = d//2 # En caso de ser None, es GC
pairs = False

basis = fixed_basis(d, num=num, pairs=pairs)
rho_1_arrays = rho_m_gen(basis, 1)
rho_2_arrays = rho_m_gen(basis, 2)
#rho_3_arrays = rho_m_gen(basis, 3)
rho_2_kkbar_arrays = rho_2_kkbar_gen(basis)
rho_2_block_arrays = rho_2_block_gen(basis, num)

100%|██████████| 12/12 [00:01<00:00,  8.86it/s]


### New

In [13]:
# New
import fermionic_mbody as fmb
basis_n = fmb.FixedBasis(d=d, num=num, pairs=pairs)
rho_1_arrays_n = fmb.rho_m_gen(basis_n, 1)
rho_2_arrays_n = fmb.rho_m_gen(basis_n, 2)
#rho_3_arrays_n = fmb.rho_m_gen(basis_n, 3)
rho_2_kkbar_arrays_n = fmb.rho_2_kkbar_gen(basis)
rho_2_block_arrays_n = fmb.rho_2_block_gen(basis)

ρ₂-block: 100%|██████████| 12/12 [00:01<00:00,  9.68it/s]


In [15]:
np.all(rho_2_block_arrays_n.todense() == rho_2_block_arrays)

True

In [5]:
# tests/test_05_pairs_kkbar.py
"""
Pair-space sanity check for rho_2_kkbar_gen (pairs=True).

For d = 4 (two time-reversed pairs) we prepare the equal superposition

    |Ψ⟩ = (|1100⟩ + |0011⟩) / √2

and verify that the **row-sums** of ρ₂_kk̄ equal the pair occupancies
⟨n_k n_{k̄}⟩ = diag ρ₁[even indices].  This identity holds for every
state because Σ_j c†_j c†_{j̄} is a completeness relation in the paired
subspace.
"""
import numpy as np

from fermionic_mbody import FixedBasis, rho_m, rho_m_gen, rho_2_kkbar_gen
from tests.conftest import dense, slater_state


def test_rho2_kkbar_row_sum_matches_rho1_diag():
    # Basis with pair compression ON
    basis = FixedBasis(d=8, num=4, pairs=True)  # size = 2

    # |Ψ⟩  =  (|1100⟩ + |0011⟩) / √2   in *paired* space
    vec_a = slater_state(basis, (0, 1, 2, 3))  # pair-0 occupied
    vec_b = slater_state(basis, (4, 5, 6, 7))  # pair-1 occupied
    psi = (vec_a + vec_b) / np.sqrt(2)

    # tensors
    rho1_t   = rho_m_gen(basis, m=1)
    rho2kk_t = rho_2_kkbar_gen(basis)

    rho1   = dense(rho_m(psi, rho1_t))        # shape (4, 4)
    rho2kk = dense(rho_m(psi, rho2kk_t))      # shape (2, 2)

    # Expected pair occupancies  ⟨n_k n_{k̄}⟩  (even indices 0,2)
    expected = rho1[::2, ::2].diagonal()      # array([0.5, 0.5])

    # Row sums must match expected occupancies
    row_sum = rho2kk.sum(axis=1)
    col_sum = rho2kk.sum(axis=0)

    print(expected, row_sum)
    assert np.allclose(row_sum, expected, atol=1e-8)
    assert np.allclose(col_sum, expected, atol=1e-8)

    # Matrix is Hermitian by construction
    assert np.allclose(rho2kk, rho2kk.conj().T, atol=1e-8)

test_rho2_kkbar_row_sum_matches_rho1_diag()

ρ₂-k k̄: 100%|██████████| 12/12 [00:00<00:00, 104.84it/s]

[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j] [0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]


In [45]:
basis = FixedBasis(d=8, num=4, pairs=True)
rho1_t   = rho_m_gen(basis, m=1)
rho1_t

ρ_1: 100%|██████████| 12/12 [00:00<00:00, 122164.19it/s]
/home/agus/anaconda3/lib/python3.11/site-packages/sparse/_utils.py:469: RuntimeWarning: invalid value encountered in scalar divide
  density = np.float64(arr.nnz) / np.float64(arr.size)
/home/agus/anaconda3/lib/python3.11/site-packages/sparse/_utils.py:487: RuntimeWarning: invalid value encountered in scalar divide
  f"{np.float64(arr.nbytes) / np.float64(reduce(operator.mul, arr.shape, 1) * arr.dtype.itemsize):.2f}"


Format,coo
Data Type,float64
Shape,"(0, 0, 6, 6)"
nnz,0
Density,nan
Read-only,True
Size,0
Storage ratio,nan
